In [1]:
!pip install -q tensorflow
!pip install -q librosa

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import librosa

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
files = list()
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        files.append(os.path.join(dirname,filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
audio_files = [file for file in files if file.endswith('.ogg')]
train_data = pd.read_csv('/kaggle/input/birdclef-2025/train.csv')
train_data.head(5)

,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license
0,1139490,[''],[''],1139490/CSA36385.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,Ragoniella pulchella,Fabio A. Sarria-S,cc-by-nc-sa 4.0
1,1139490,[''],[''],1139490/CSA36389.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,Ragoniella pulchella,Fabio A. Sarria-S,cc-by-nc-sa 4.0
2,1192948,[''],[''],1192948/CSA36358.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0
3,1192948,[''],[''],1192948/CSA36366.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.2800,-73.8582,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0
4,1192948,[''],[''],1192948/CSA36373.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0


In [4]:
train_data.describe()

,rating,latitude,longitude
count,28564.000000,27755.000000,27755.000000
mean,2.917063,-0.533480,-68.524380
std,1.964896,17.609276,21.614566
min,0.000000,-54.857400,-163.680000
25%,0.000000,-15.084600,-79.649200
50%,4.000000,1.131600,-73.495400
75%,4.500000,9.511000,-53.801800
max,5.000000,68.374800,153.551400


In [5]:
train_data.scientific_name.value_counts()

scientific_name
Pitangus sulphuratus      990
Nyctidromus albicollis    808
Tyrannus melancholicus    787
Rupornis magnirostris     709
Coereba flaveola          610
                         ... 
Elachistocleis pearsei      2
Bradypus variegatus         2
Pecari tajacu               2
Cerdocyon thous             2
Ragoniella pulchella        2
Name: count, Length: 206, dtype: int64

In [6]:
train_data.common_name.value_counts()

common_name
Great Kiskadee                     990
Common Pauraque                    808
Tropical Kingbird                  787
Roadside Hawk                      709
Bananaquit                         610
                                  ... 
Colombian Plump Frog                 2
Brown-throated Three-toed Sloth      2
Collared Peccary                     2
Crab-eating Fox                      2
Ragoniella pulchella                 2
Name: count, Length: 206, dtype: int64

In [25]:
train_filename = [os.path.basename(file) for file in train_data.filename]
train_filename[28560:]

['iNat78624.ogg', 'iNat789234.ogg', 'iNat819873.ogg', 'iNat922688.ogg']

In [27]:
audio_set = sorted(audio_files)
sel_audio = audio_set[:28564]
sel_audio[28560:]

['/kaggle/input/birdclef-2025/train_audio/ywcpar/iNat78624.ogg',
 '/kaggle/input/birdclef-2025/train_audio/ywcpar/iNat789234.ogg',
 '/kaggle/input/birdclef-2025/train_audio/ywcpar/iNat819873.ogg',
 '/kaggle/input/birdclef-2025/train_audio/ywcpar/iNat922688.ogg']

In [15]:
def matching_files(audio,train_files):
    """Returns the matching files in the train set"""
    train_set = set(train_files)
    common = [audio_file for file in train_set for audio_file in audio if file.split('/')[-1] == audio_file.split('/')[-1]]
    return common

In [28]:
train_data['audio_path'] = sel_audio

In [12]:
import tensorflow as tf
import tensorflow_io as tfio

In [13]:
def convert_to_audio_tensor(files):
    audio_tensors = dict()
    for file in train_audio_data:
        audio = tfio.audio.AudioIOTensor(file)
        audio_tensors[os.path.basename(file)] = audio
    return audio_tensors

In [ ]:
import tensorflow as tf

class preprocessing:
    def __init__(self,audio_files):
        self.audio_files = audio_files
        
    def normalize(self,audio_file):
        """Returns the normalized array of audio files"""
        normalized_arr = dict()
        for k,v in audio_file.items():
            tensor = tf.cast(v,tf.float32) / 32767.0
            normalized_arr[k] = tensor
        return normalized_arr
        
    def trim_audio(self,audio_file):
        """Returns the trim audio version """
        trimmed_audio = dict()
        for k,v in audio_file:
            trimmed = tfio.audio.trim(v,axis=0,epsilon=0.2)
            start,stop = trimmed[0],trimmed[1]
            audio = v[start:stop]
            trimmed_audio[k] = audio
        return trimmed_audio
    
    def preprocessing_audio_files(self):
        """Returns the audio files"""
        processed_files = dict()
        v = audio_file[k]
        fade_output = tfio.audio.fade(v,fade_in=1000,fade_out=2000,mode='logarithmic')
        processed_files[k] = fade_output

        return processed_files

In [ ]:
processed_audio_files = preprocessing_audio_files(audio_tensors)

In [ ]:
def spectogram(processed_files):
    """Returns the spectogram of the processed audio files"""

    for k,v in processed_files:
        spectogram = tfio.audio.spectogram(v,nfft=512,window=512,stride=256)
        mel_spectogram = tfio.audio.melscale(spectogram,rate=16000,mels=128,fmin=0,fmax=8000)
        db_scale = tfio.audio.dbscale(mel_spectogram,top_db=80)
        